# Bagging

Razvijemo razred za tehniko bagging, oziroma [bootstrap aggregating](https://en.wikipedia.org/wiki/Bootstrap_aggregating), ki lahko izboljša stabilnost in točno napovednih metod, katerih struktura modela je lahko precej odvisna od vzorca podatkov. To je, katerih struktura zgrajenega modela se pri manjših spremembah v učnih podatkih lahko precej spremeni. Primer take metode so klasifikacijska in regresijska drevesa, lahko pa bagging pomaga tudi kakšnim drugim tehnikam. Bagging si ogledamo tudi kot primer ansamblov, torej metod, ki temeljijo na gradnji množice modelov in potem s povprečenjem ali glasovanjem predlagajo vrednost razreda.

Najprej preberemo podatke.

In [1]:
import Orange
from Orange.preprocess.preprocess import Preprocess
import numpy as np
import random

In [2]:
class RemoveConstant(Preprocess):
    def __call__(self, data):
        oks = np.min(data.X, axis=0) != np.max(data.X, axis=0)
        atts = [a for a, ok in zip(data.domain.attributes, oks) if ok]
        domain = Orange.data.Domain(atts, data.domain.class_vars)
        return Orange.data.Table(domain, data)

rc = RemoveConstant()

In [3]:
orig = Orange.data.Table("data/smoking-small.tab")
data = rc(orig)
len(data), len(orig.domain.attributes), len(data.domain.attributes), len(data.domain.class_var.values)

(79, 1000, 1000, 2)

Na podatkih ovrednotimo nekaj osnovnih klasifikatorjev. Uporabimo klasifikacijska drevesa in k-najbližjih sosedov, za vsak slučaj pa preverjamo še napovedovanje z večinskim razredom, pri katerem bi moral biti AUC enak 0.5.

In [4]:
maj = Orange.classification.MajorityLearner()
tree = Orange.classification.TreeLearner()
knn = Orange.classification.KNNLearner()

In [5]:
learners = [maj, tree, knn]
res = Orange.evaluation.CrossValidation(data, learners, k=5, random_state=42)

In [6]:
print("\n".join("%20s %5.2f" % (learner.name, score) for learner, score in zip(learners, Orange.evaluation.AUC(res))))

            majority  0.50
                tree  0.72
                 knn  0.77


Drevesa se ne obneseje najbolje, nekoliko boljši so najbližji sosedje.

### Orange-ov razred za Bagging

Orange-ova shema za napovedne modele vedno vsebuje Learner in Model, torej, razred, ki je namenjen učenju in razred, ki je namenjen napovedovanju. Uporabimo razred in ne funkcije: iz njih izvedeni objekti si bodo morali zapomniti nekatere nastavitve in parametre, potem pa jih bomo uporabili pri učenju in napovedovanju. Razred za učenje (BaggedLearner) si bo moral zapomniti algoritem učenja, ki ga bomo uporabili na k vzorcih podatkov, razred za napovedovanje (BaggedModel) pa bomo inicializirali s seznamom k naučenih modelov.

BaggedLearner implementira vzorčenje po metodi stremena (angl. boosting). Po inicializaciji objekt kličemo s podatki. Klic nam vrne regresor, BaggedModel, ki hrani naučene modele. Ob klicu regresorja ta uporabi napovedne modele in izračuna povprečje napovedi, ki jih ti vrnejo. BaggedModel lahko kličemo z enim samim primerom, lahko pa mu tudi podtaknemo tabelo primerov. Pri slednjem nam bo klic tega objekta vrnil vektor napovedi.

Bagging bomo implementirali za regresijo, torej privzeli, da nam napovedni modeli vračajo realne vrednost. Pri klasifikaciji bi implementacija bila rahlo kompleksnejša, saj nam v Orange-u klasifikatorji lahko vračajo razrede ali pa njihove verjetnosti, odvisno od parametra, ki ga podamo pri klicu klasifikatorja.

In [7]:
class BaggedLearner(Orange.classification.Learner):
    """Bootstrap aggregating learner."""
    def __init__(self, learner, k=3):
        super().__init__()
        self.k = k # number of bootstrap samples and corresponding models
        self.learner = learner # base learner
        self.name = "bagged " + self.learner.name
    
    def fit_storage(self, data):
        """Return a bagged model inferred from the training set."""
        models = []
        for epoch in range(self.k):
            indices = np.random.randint(len(data), size=len(data))  # sample indices
            models.append(self.learner(data[indices]))  # model inference from a sample
        model = BaggedModel(data.domain, models)  # return a predictor that remembers a list of models
        model.name = self.name
        return model

class BaggedModel(Orange.classification.Model):
    """Bootstrap aggregating classifier."""
    def __init__(self, domain, models):
        super().__init__(domain)
        self.models = models  # a list of predictors
    
    def predict_storage(self, data, ret=Orange.classification.Model.Value):
        """Given a data instance or table of data instances returns predicted class."""
        p = np.mean([m(data, 1) for m in self.models], axis=0)
        return p / p.sum(axis=1, keepdims=True)

Uporabimo bagging nad drevesi in linearno regresijo. Točnost napovedi ocenimo s prečnim preverjanjem.

In [8]:
bag_tree = BaggedLearner(tree, k=10)
bag_knn = BaggedLearner(knn, k=10)

In [9]:
bt = bag_tree(data)
bt(data[0], 1)

array([[ 0.00529617,  0.99470383]])

In [10]:
bt(data[0:3], 1)

array([[ 0.00529617,  0.99470383],
       [ 0.00529617,  0.99470383],
       [ 0.10285714,  0.89714286]])

In [11]:
learners = [tree, bag_tree, knn, bag_knn]
# learners = [tree, bag_tree]
res = Orange.evaluation.CrossValidation(data, learners, k=5, random_state=42)

In [12]:
print("\n".join("%20s %5.2f" % (learner.name, score) for learner, score in zip(learners, Orange.evaluation.AUC(res))))

                tree  0.72
         bagged tree  0.87
                 knn  0.77
          bagged knn  0.76


Bagging z drevesi se med zgornjimi dobro obnese. Izboljšanje najbližjih sosedov pa nam po drugi strani ni uspelo. Zakaj? Drevesa zgrajena na vzorčenih podatki se dejansko bistveno razlikujejo med sabo (kako bi to preverili?), razlike med dobljenimi modeli najbližjih sosedov pa so majhne.

# Ponaključenje

Sledi finta. V angleščini bi se ji reklo *randomization*. Namreč, radi bi izboljšali tudi napovedi najbližjih sosedov. In pri tem uporabili ansamble, oziroma niz napovednih modelov. Ampak, kot že rečeno, ti napovedni modeli se morajo med sabo razlikovati, sicer nam bagging prav dosti ne bo pomagal. Da bi modeli tudi pri stabilnih klasifikatorjih, kot so najbližji sosedi, med sabo bili čimbolj različni, lahko namesto vzorčenja podatkov vzorčimo atribute.

Spodaj implementiramo samo razred za učenje (RandomizedLearner), ki gradimo modele na vzorčenih množicah atributov, za napovedovanje s povprečenjem napovedi večih modelov pa recikliramo kar BaggedModel. RandomizedLearner je podobno preprost kot BaggedLearner. Za izbor atributov oziroma konstrukcijo nove tabele z podmnožico originalnih atributov smo uporabili kar Orange-ov SelectRandomFeatures.

In [13]:
class RandomizedLearner(Orange.classification.Learner):
    """Ensamble learning through randomization of data domain."""
    def __init__(self, learner, k=3, p=0.5):
        super().__init__()
        self.k = k
        self.learner = learner
        self.name = "rand " + self.learner.name
        # a function to be used for random attribute subset selection
        self.selector = Orange.preprocess.fss.SelectRandomFeatures(k=p)
    
    def fit_storage(self, data):
        """Returns a bagged model with randomized regressors."""
        models = []
        for epoch in range(self.k):
            sample = self.selector(data) # data with a subset of attributes
            models.append(self.learner(sample))
        model = BaggedModel(data.domain, models)
        model.name = self.name
        return model

In [15]:
rnd_knn = RandomizedLearner(knn, k=10, p=0.2)
learners = [maj, knn, rnd_knn]
res = Orange.evaluation.CrossValidation(data, learners, k=5, random_state=42)

In [16]:
print("\n".join("%20s %5.2f" % (learner.name, score) for learner, score in zip(learners, Orange.evaluation.AUC(res))))

            majority  0.50
                 knn  0.77
            rand knn  0.84


Tole deluje še kar ok, oziroma logistične regresije vsaj ne pokvari. Na kakšni drugi domeni bi lahko tudi pomagalo, oziroma se zna zgoditi, da ima ansambel ponaključenih logističnih regresij večjo točnost od logistične regresije same.

# Naključni gozd

Klasičen, ali morda celo najbolj tipičen primer ponaklučenja (randomizacije) so naključni gozdovi (angl. random forest). Tu je bil narejen poseg v algoritem učenja dreves, ki ob gradnji drevesa ne uporabi vač za razcep množice primerov najprimernejšega atributa, ampak atribut za razcep naključno izbere iz množice najprimernejši. Poleg uporabljenega vzorčanje z metodo stremena to še dodatno poskrbi za raznolikost dreves v gozdu.

Spodaj smo kot primer uporabili kar SimpleRandomForestLearner, ki je hitra C-jevska implementacija naključnih gozdov. Dokler ni scikit-learn odpravil nekaj bugov in pospešil svojo implementacijo, je bil SimpleRandomForestLearner morda najhitrejša implementacija naključnega gozda z vmesnikom v Pythonu. Orange zawrapa sicer tudi implementacijo iz sklearn-a (RandomForestLearner), ki pa meni, vsaj na tej domeni, včasih daje malce slabše rezultate od Orange-ovega.

In [17]:
forest = Orange.classification.RandomForestLearner(n_estimators=10)
forest.name = "skl forest"
learners = [tree, forest]
res = Orange.evaluation.CrossValidation(data, learners, k=5, random_state=42)

In [18]:
print("\n".join("%20s %5.2f" % (learner.name, score) for learner, score in zip(learners, Orange.evaluation.AUC(res))))

                tree  0.72
          skl forest  0.86


Še boljše rezultate dobimo, če dvignemo število dreves na, recimo 500. Tipično se pri naključnih gozdovih gradi recimo 500 ali 1000 dreves, večje število dreves od teh pa nam navadno ne pomaga kaj dosti.

In [19]:
forest.params['n_estimators'] = 500
learners = [tree, forest]
res = Orange.evaluation.CrossValidation(data, learners, k=5, random_state=42)

In [20]:
print("\n".join("%20s %5.2f" % (learner.name, score) for learner, score in zip(learners, Orange.evaluation.AUC(res))))

                tree  0.72
          skl forest  0.96
